
# `upstox_v3_18Sep_clean.ipynb` — Intraday Index Options (NIFTY / BANKNIFTY / FINNIFTY)

**Purpose**: Clean, reproducible "Run‑All" notebook to stream Upstox v3 market data for **weekly index options**, select a compact strike set, validate an (LLM/mocked) strategy plan, and **simulate** order placement + exits by default (no live orders).

**Key Defaults (safe)**
- `SIMULATION_MODE = True` — run a deterministic synthetic feed, no Upstox connectivity required.
- `USE_LLM = False` — use a mocked plan (deterministic JSON).
- `ORDERS_LIVE = False` — always **dry-run** orders by default.
- `EXIT_MANAGER_LIVE = False` — simulate exits; no live modifications.

> Flip these flags **consciously** for real trading. The notebook is structured so you can switch to live feed and order placement by toggling the four flags in a single cell.



## Environment & requirements

- Python 3.9+ recommended
- Packages: `pandas`, `numpy`, `nbformat` (for this generated file only), optionally `upstox_client` for live mode.
- If you intend to trade live, ensure you have:
  - Upstox v3 **access token**
  - Proper **product mapping** (e.g., intraday = `I`) and **lot sizes**
  - Exchange/SEBI-compliant controls enabled

> **Note**: This notebook avoids indefinite loops in **simulation mode** so "Run‑All" finishes. In **live mode** the streamer thread runs until you stop it.


In [7]:

# ---- Imports & global setup ----
import os, json, time, threading, math, random
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import numpy as np
import pandas as pd

pd.set_option("display.width", 160)
pd.set_option("display.max_columns", 60)

# Try importing Upstox SDK, keep notebook runnable without it
try:
    import upstox_client
    UPSDK_AVAILABLE = True
except Exception as e:
    UPSDK_AVAILABLE = False
    print("Upstox SDK not available. Live streaming & live orders are disabled until you install and configure it.")
    print("To install: pip install upstox-python-sdk  # (package name may vary; see Upstox docs)")


In [8]:

# ---- Safety toggles & config ----
SIMULATION_MODE = False      # <== Set False for live Upstox websocket + instruments
USE_LLM = True             # <== Set True to call your LLM; otherwise uses a mocked plan
ORDERS_LIVE = True         # <== Set True to place live orders
EXIT_MANAGER_LIVE = True   # <== Set True to manage exits live

UNDERLYING = "NIFTY"        # Allowed: "NIFTY", "BANKNIFTY", "FINNIFTY"
SPAN_STRIKES = 2            # span=2 -> 5 strikes total around ATM
WEBSOCKET_MODE = "full_d30" # Upstox mode used in live streaming
RUN_DEMO_SECONDS = 10       # how long to run the synthetic feed demo

# Per-underlying strike step
STRIKE_STEP = {"NIFTY": 50, "BANKNIFTY": 100, "FINNIFTY": 50}

# Risk limits
MAX_QTY_PER_LEG = 300       # hard cap for per-leg quantity
MAX_OPEN_LEGS = 6           # cap number of concurrent legs

# Timezone helpers
IST = "Asia/Kolkata"


In [9]:

# ---- Credentials & product mapping ----
class CredentialUpstox:
    # Prefer environment variable; fallback to manual paste
    ACCESS_TOKEN = os.getenv("UPSTOX_ACCESS_TOKEN", "")  # paste token if not set

PRODUCT_MAP = {
    # Logical -> Upstox product code (confirm with Upstox docs)
    "MIS": "I",   # Intraday
    "NRML": "D",  # Carry/Delivery
}

# Safety: for FO index options we typically use Intraday ("I") by default
DEFAULT_PRODUCT = "MIS"


In [10]:

# ---- Utilities ----
WEEKLY_DOW = {"NIFTY": 1, "BANKNIFTY": 3, "FINNIFTY": 1}  # Mon=0, Tue=1, ..., Thu=3

def next_weekly_expiry(underlying: str, today=None) -> str:
    today = pd.Timestamp.now(IST).normalize() if today is None else pd.Timestamp(today, tz=IST).normalize()
    target = WEEKLY_DOW.get(underlying.upper(), 3)
    days_ahead = (target - today.weekday()) % 7
    if days_ahead == 0:
        days_ahead = 7
    return (today + pd.Timedelta(days=days_ahead)).strftime("%Y-%m-%d")

def nearest_strikes(ltp: float, underlying: str, span: int = 2) -> List[int]:
    step = STRIKE_STEP.get(underlying.upper(), 50)
    # round-half-up
    nearest = int(step * math.floor((ltp + step/2) / step))
    return [nearest + i * step for i in range(-span, span+1)]  # inclusive

def to_ist_ms(ms) -> pd.Timestamp:
    try:
        return pd.to_datetime(int(ms), unit="ms", utc=True).tz_convert(IST)
    except Exception:
        return pd.NaT


In [11]:

# ---- Instruments master (df_futureOptions) ----
def build_simulated_instruments(underlying: str, base_atm: int, span: int, step_map: Dict[str, int]) -> pd.DataFrame:
    expiry = next_weekly_expiry(underlying)
    step = step_map.get(underlying.upper(), 50)
    strikes = [base_atm + i*step for i in range(-span*2, span*2+1)]  # wider set to allow filter
    rows = []
    tok = 60000
    lot_size = 50 if underlying.upper() == "NIFTY" else (15 if underlying.upper() == "BANKNIFTY" else 40)
    for sp in strikes:
        for opt in ("CE","PE"):
            tok += 1
            rows.append({
                "weekly": 1.0,
                "segment": "NSE_FO",
                "name": underlying.upper(),
                "exchange": "NSE",
                "expiry": expiry,
                "instrument_type": opt,
                "asset_symbol": underlying.upper(),
                "underlying_symbol": underlying.upper(),
                "instrument_key": f"NSE_FO|{tok}",
                "lot_size": float(lot_size),
                "freeze_quantity": 18000.0,
                "exchange_token": tok,
                "minimum_lot": 1.0,
                "tick_size": 0.05,
                "asset_type": "IDX",
                "underlying_type": "IDX",
                "trading_symbol": f"{underlying.upper()} {sp} {opt} {pd.to_datetime(expiry).strftime('%d %b %y').upper()}",
                "strike_price": float(sp),
                "qty_multiplier": 1.0,
                "isin": None,
                "security_type": "OPTIDX",
                "short_name": None,
                "asset_key": None,
                "underlying_key": None,
                "last_trading_date": expiry,
                "price_quote_unit": "INR"
            })
    return pd.DataFrame(rows)

def load_instruments(underlying: str, simulation: bool = True) -> pd.DataFrame:
    if simulation:
        # For simulation, assume an ATM around 24000 for NIFTY, 50500 for BANKNIFTY, 22000 for FINNIFTY
        base_atm = 24000 if underlying.upper() == "NIFTY" else (50500 if underlying.upper() == "BANKNIFTY" else 22000)
        df = build_simulated_instruments(underlying, base_atm, span=SPAN_STRIKES, step_map=STRIKE_STEP)
        return df
    # Live path: download NSE instruments; requires internet
    url = "https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz"
    try:
        df = pd.read_json(url)
    except Exception as e:
        raise RuntimeError(f"Failed to load instruments from {url}: {e}")
    # Normalize
    if "expiry" in df:
        df["expiry"] = pd.to_datetime(df["expiry"], unit="ms", errors="coerce").dt.strftime("%Y-%m-%d")
    if "strike_price" in df:
        df["strike_price"] = pd.to_numeric(df["strike_price"], errors="coerce")
    if "lot_size" in df:
        df["lot_size"] = pd.to_numeric(df["lot_size"], errors="coerce")
    return df

df_futureOptions = load_instruments(UNDERLYING, simulation=SIMULATION_MODE)
assert not df_futureOptions.empty, "Instrument master is empty"

display(df_futureOptions.head(6))


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
0,0.0,NCD_FO,JPYINR,NSE,2026-03-27,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,10000.0,14294,1.0,0.25,CUR,CUR,JPYINR 61 CE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NCD_FO,JPYINR,NSE,2026-03-27,PE,JPYINR,JPYINR,NCD_FO|14295,1.0,10000.0,14295,1.0,0.25,CUR,CUR,JPYINR 61 PE 27 MAR 26,61.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NSE_EQ,SDL RJ 7.49% 2035,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250163,100.0,100000.0,758718,NaN,1.00,NaN,NaN,749RJ35,NaN,1.0,IN2920250163,NORMAL,NaN,NaN,NaN,NaN,NaN
3,NaN,NSE_EQ,SDL RJ 7.57% 2043,NSE,NaN,SG,NaN,NaN,NSE_EQ|IN2920250171,100.0,100000.0,758723,NaN,1.00,NaN,NaN,757RJ43,NaN,1.0,IN2920250171,NORMAL,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,GBPINR,NSE,2025-12-29,PE,GBPINR,GBPINR,NCD_FO|14277,1.0,10000.0,14277,1.0,0.25,CUR,CUR,GBPINR 118.25 PE 29 DEC 25,118.25,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,NCD_FO,GBPINR,NSE,2025-12-29,CE,GBPINR,GBPINR,NCD_FO|14274,1.0,10000.0,14274,1.0,0.25,CUR,CUR,GBPINR 118 CE 29 DEC 25,118.00,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:

# ---- Build current weekly option chain for the UNDERLYING ----
expiry_target = next_weekly_expiry(UNDERLYING)

df_chain = df_futureOptions[
    (df_futureOptions["segment"] == "NSE_FO") &
    (df_futureOptions["name"].str.upper() == UNDERLYING.upper()) &
    (df_futureOptions["instrument_type"].isin(["CE","PE"])) &
    (df_futureOptions["expiry"] == expiry_target)
].copy()

# Validate non-empty
assert not df_chain.empty, f"No options found for {UNDERLYING} expiry={expiry_target}"

print(f"Chain size for {UNDERLYING} weekly {expiry_target}: {len(df_chain)} rows")

# We'll compute strikes after we have an LTP (simulated or live). For simulation we set an initial LTP:
if SIMULATION_MODE:
    # Set a synthetic underlying last price to anchor strikes
    init_ltp = df_chain["strike_price"].median()
else:
    init_ltp = None  # to be set after reading index spot feed (not covered here)

display(df_chain.head(8))


Chain size for NIFTY weekly 2025-09-23: 180 rows


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
16844,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47723,75.0,1800.0,47723,75.0,5.0,INDEX,INDEX,NIFTY 24950 CE 23 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16847,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47724,75.0,1800.0,47724,75.0,5.0,INDEX,INDEX,NIFTY 24950 PE 23 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16848,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47718,75.0,1800.0,47718,75.0,5.0,INDEX,INDEX,NIFTY 24900 PE 23 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16849,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47717,75.0,1800.0,47717,75.0,5.0,INDEX,INDEX,NIFTY 24900 CE 23 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16850,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47712,75.0,1800.0,47712,75.0,5.0,INDEX,INDEX,NIFTY 24850 PE 23 SEP 25,24850.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16851,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47711,75.0,1800.0,47711,75.0,5.0,INDEX,INDEX,NIFTY 24850 CE 23 SEP 25,24850.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16864,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47752,75.0,1800.0,47752,75.0,5.0,INDEX,INDEX,NIFTY 25050 PE 23 SEP 25,25050.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16865,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47751,75.0,1800.0,47751,75.0,5.0,INDEX,INDEX,NIFTY 25050 CE 23 SEP 25,25050.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN


In [ ]:

# ---- Feed dataframes & streaming scaffolding ----

# Global shared DataFrames
df_feed = pd.DataFrame(columns=[
    "Token","Ltp","Ltq","Cp","BidP1","BidQ1","AskP1","AskQ1","Ltt","Oi","Iv","Atp","Tbq","Tsq"
])
df_feed_enriched = pd.DataFrame()

_df_lock = threading.Lock()

def enrich_feed(_df_feed: pd.DataFrame, _df_meta: pd.DataFrame,
                cols_to_add=None) -> pd.DataFrame:
    if cols_to_add is None:
        cols_to_add = ["lot_size","trading_symbol","strike_price","tick_size","instrument_type","expiry","name"]
    left = _df_feed.copy()
    right = _df_meta[["instrument_key"] + [c for c in cols_to_add if c in _df_meta.columns]].drop_duplicates("instrument_key")
    out = left.merge(right, left_on="Token", right_on="instrument_key", how="left", validate="m:1")
    # Microstructure
    out["Mid"] = np.where(out["BidP1"].notna() & out["AskP1"].notna(), (out["BidP1"] + out["AskP1"])/2, out["Ltp"])
    out["Spread"] = np.where(out["BidP1"].notna() & out["AskP1"].notna(), (out["AskP1"] - out["BidP1"]), np.nan)
    return out

# ---- Simulation streamer ----
class SimStreamer(threading.Thread):
    def __init__(self, tokens: List[str], step: float = 0.5, period_s: float = 0.5, run_seconds: int = 10):
        super().__init__(daemon=True)
        self.tokens = tokens
        self.step = step
        self.period_s = period_s
        self.run_seconds = run_seconds
        self._stop_evt = threading.Event()
        # seed initial prices around strike +/- random offset
        self._state = {t: float(df_chain.loc[df_chain["instrument_key"]==t,"strike_price"].iloc[0]) * (1 + random.uniform(-0.02, 0.02))
                       for t in tokens}

    def stop(self):
        self._stop_evt.set()

    def run(self):
        start = time.time()
        lvl = 30  # simulate up to 30 levels, but we'll only use top-of-book here
        while not self._stop_evt.is_set():
            now = time.time()
            if now - start > self.run_seconds:
                break
            for tok in self.tokens:
                # random walk
                p = self._state[tok]
                p += random.choice([-1,1]) * self.step * random.random()
                self._state[tok] = max(0.5, p)
                ltp = round(self._state[tok], 2)
                bid = round(max(0.05, ltp - 0.35), 2)
                ask = round(ltp + 0.35, 2)
                bidq = random.choice([150,300,450,600,900])
                askq = random.choice([150,300,450,600,900])
                ltq = random.choice([75,150,225])
                cp = round(random.uniform(-100, 100), 2)
                oi = random.choice([100000, 250000, 500000])
                iv = round(random.uniform(0.12, 0.28), 5)
                atp = round(ltp - random.uniform(-10, 10), 2)
                tbq = random.randint(10000, 500000)
                tsq = random.randint(10000, 500000)
                ltt = int(pd.Timestamp.now(IST).tz_convert("UTC").value / 1_000_000)  # ms epoch

                row = {
                    "Token": tok, "Ltp": ltp, "Ltq": float(ltq), "Cp": cp,
                    "BidP1": bid, "BidQ1": float(bidq),
                    "AskP1": ask, "AskQ1": float(askq),
                    "Ltt": to_ist_ms(ltt), "Oi": float(oi), "Iv": float(iv),
                    "Atp": float(atp), "Tbq": float(tbq), "Tsq": float(tsq),
                }

                with _df_lock:
                    global df_feed, df_feed_enriched
                    if tok in df_feed["Token"].values:
                        for k,v in row.items():
                            df_feed.loc[df_feed["Token"]==tok, k] = v
                    else:
                        df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)
                    df_feed_enriched = enrich_feed(df_feed, df_chain)
            time.sleep(self.period_s)

# ---- Live streamer scaffolding (only used if SIMULATION_MODE=False) ----
def start_live_stream(tokens: List[str], mode: str = "full_d30"):
    if not UPSDK_AVAILABLE:
        raise RuntimeError("Upstox SDK not installed. Install and configure to use live streaming.")
    if not CredentialUpstox.ACCESS_TOKEN:
        raise RuntimeError("ACCESS_TOKEN missing. Set UPSTOX_ACCESS_TOKEN or paste into CredentialUpstox.ACCESS_TOKEN")

    configuration = upstox_client.Configuration()
    configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    api_client = upstox_client.ApiClient(configuration)

    streamer = upstox_client.MarketDataStreamerV3(api_client, instrument_key=tokens, mode=mode)

    def _on_message(msg):
        # Expected msg similar to provided sample structure
        feeds = msg.get("feeds", {})
        for token, payload in feeds.items():
            ff = payload.get("fullFeed",{}).get("marketFF",{})
            ltpc = ff.get("ltpc",{})
            level = ff.get("marketLevel",{}).get("bidAskQuote",[{}])
            ohlc = ff.get("marketOHLC",{}).get("ohlc",[])

            row = {
                "Token": token,
                "Ltp": float(ltpc.get("ltp")) if ltpc.get("ltp") is not None else np.nan,
                "Ltq": float(ltpc.get("ltq")) if ltpc.get("ltq") is not None else np.nan,
                "Cp": float(ltpc.get("cp")) if ltpc.get("cp") is not None else np.nan,
                "BidP1": float(level[0].get("bidP")) if level and level[0].get("bidP") is not None else np.nan,
                "BidQ1": float(level[0].get("bidQ")) if level and level[0].get("bidQ") is not None else np.nan,
                "AskP1": float(level[0].get("askP")) if level and level[0].get("askP") is not None else np.nan,
                "AskQ1": float(level[0].get("askQ")) if level and level[0].get("askQ") is not None else np.nan,
                "Ltt": to_ist_ms(ltpc.get("ltt")),
                "Oi": float(ff.get("oi")) if ff.get("oi") is not None else np.nan,
                "Iv": float(ff.get("iv")) if ff.get("iv") is not None else np.nan,
                "Atp": float(ff.get("atp")) if ff.get("atp") is not None else np.nan,
                "Tbq": float(ff.get("tbq")) if ff.get("tbq") is not None else np.nan,
                "Tsq": float(ff.get("tsq")) if ff.get("tsq") is not None else np.nan,
            }
            with _df_lock:
                global df_feed, df_feed_enriched
                if token in df_feed["Token"].values:
                    for k,v in row.items():
                        df_feed.loc[df_feed["Token"]==token, k] = v
                else:
                    df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)
                df_feed_enriched = enrich_feed(df_feed, df_chain)

    def _on_open():
        print("WebSocket opened")

    def _on_error(err):
        print("WebSocket error:", err)

    def _on_close():
        print("WebSocket closed")

    streamer.on_message = _on_message
    streamer.on_open = _on_open
    streamer.on_error = _on_error
    streamer.on_close = _on_close

    streamer.connect()  # non-blocking in SDK; if blocking, consider threading
    return streamer


In [14]:

# ---- Strategy plan (LLM or mock) & validation ----

def ask_llm_for_strategy(df_snapshot: pd.DataFrame, use_mock: bool = True) -> Dict[str, Any]:
    """Return a plan dict with a list of legs: token, side, qty, product, order_type, price(optional)."""
    if use_mock:
        # Simple mocked plan: Short strangle at nearest CE/PE (qty = lot_size)
        if df_snapshot.empty:
            return {"legs": []}
        # Pick ATM by nearest to Mid
        snap = df_snapshot.dropna(subset=["Mid","strike_price","instrument_type"]).copy()
        snap["dist"] = (snap["Mid"] - snap["strike_price"]).abs()
        atm = snap.sort_values(["dist"]).groupby("instrument_type").head(1)
        legs = []
        for _, r in atm.iterrows():
            lot = int(r["lot_size"]) if pd.notna(r["lot_size"]) else 0
            qty = lot if lot > 0 else 50
            side = "SELL"  # short the ATM strangle
            legs.append({"token": str(r["Token"]), "side": side, "qty": int(qty), "product": DEFAULT_PRODUCT, "order_type": "MARKET"})
        return {"legs": legs}
    else:
        # Placeholder: Integrate your LLM call here, must return the same schema
        raise NotImplementedError("LLM path is disabled by default. Set USE_LLM=True and implement the call.")

def validate_strategy(plan: Dict[str, Any], df_enriched: pd.DataFrame) -> Optional[Dict[str, Any]]:
    if not plan or "legs" not in plan or not isinstance(plan["legs"], list):
        raise ValueError("Plan must contain a 'legs' list")
    vlegs = []
    used_tokens = set()
    for leg in plan["legs"]:
        token = str(leg.get("token",""))
        if not token:
            raise ValueError("Missing token in leg")
        if token in used_tokens:
            raise ValueError(f"Duplicate token in plan: {token}")
        used_tokens.add(token)

        side = leg.get("side","").upper()
        if side not in {"BUY","SELL"}:
            raise ValueError(f"Invalid side: {side}")

        try:
            qty = int(leg.get("qty", 0))
        except Exception:
            raise ValueError(f"Invalid qty for token {token}")
        if qty <= 0 or qty > MAX_QTY_PER_LEG:
            raise ValueError(f"Qty out of bounds for token {token}: {qty}")

        # Validate instrument exists and is an option on our underlying
        row = df_enriched.loc[df_enriched["Token"]==token]
        if row.empty:
            raise ValueError(f"Token not present in current feed/meta: {token}")
        row = row.iloc[0]
        if str(row.get("instrument_type")) not in {"CE","PE"}:
            raise ValueError(f"Token {token} is not CE/PE")
        if str(row.get("name","")).upper() != UNDERLYING.upper():
            raise ValueError(f"Token {token} underlying mismatch: {row.get('name')} != {UNDERLYING}")

        product = leg.get("product", DEFAULT_PRODUCT)
        if product not in PRODUCT_MAP:
            raise ValueError(f"Unknown product: {product}")

        order_type = leg.get("order_type","MARKET")
        if order_type not in {"MARKET","LIMIT"}:
            raise ValueError(f"Unsupported order_type: {order_type}")

        vlegs.append({
            "token": token,
            "side": side,
            "qty": qty,
            "product": product,
            "order_type": order_type,
            "price": float(leg.get("price", 0)) if order_type=="LIMIT" else None,
        })
    if len(vlegs) > MAX_OPEN_LEGS:
        raise ValueError(f"Too many legs: {len(vlegs)} > {MAX_OPEN_LEGS}")
    return {"legs": vlegs}


In [15]:

# ---- Order placement & Exit manager ----
open_positions = {}  # token -> dict(side, qty, avg_price)

def record_fill(fill: Dict[str, Any]):
    token = str(fill["token"])
    side = fill["side"].upper()
    qty = int(fill["qty"])
    price = float(fill.get("price", 0.0))
    pos = open_positions.get(token, {"qty": 0, "side": side, "avg_price": 0.0})
    # For simplicity, average in by qty
    if pos["qty"] == 0:
        pos = {"qty": qty, "side": side, "avg_price": price}
    else:
        new_qty = pos["qty"] + qty if side == pos["side"] else pos["qty"] - qty
        if new_qty <= 0:
            pos = {"qty": 0, "side": side, "avg_price": 0.0}
        else:
            pos = {"qty": new_qty, "side": side, "avg_price": (pos["avg_price"]*pos["qty"] + price*qty)/max(new_qty,1)}
    open_positions[token] = pos

@dataclass
class ExitConfig:
    dry_run: bool = True
    target_pct: float = 0.2   # 20% profit target
    stop_pct: float = 0.4     # 40% stop
    check_interval_s: float = 1.0

_exit_thread = None
_exit_stop_evt = threading.Event()

def should_exit_position(token: str, row: pd.Series, pos: Dict[str, Any], cfg: ExitConfig) -> Optional[Dict[str, Any]]:
    # Use Mid price if available
    px = row.get("Mid", np.nan)
    if np.isnan(px):
        px = row.get("Ltp", np.nan)
    if np.isnan(px) or pos["qty"] <= 0:
        return None

    entry = pos["avg_price"]
    side = pos["side"]
    # PnL per unit for long = (px - entry), for short = (entry - px)
    pnl = (px - entry) if side=="BUY" else (entry - px)
    if entry <= 0:
        return None
    pnl_pct = pnl / entry

    if pnl_pct >= cfg.target_pct:
        return {"action":"EXIT","reason":"target","token":token,"qty":pos["qty"],"side": "SELL" if side=="BUY" else "BUY"}
    if pnl_pct <= -cfg.stop_pct:
        return {"action":"EXIT","reason":"stop","token":token,"qty":pos["qty"],"side": "SELL" if side=="BUY" else "BUY"}
    return None

def _exit_worker(cfg: ExitConfig):
    while not _exit_stop_evt.is_set():
        time.sleep(cfg.check_interval_s)
        with _df_lock:
            snapshot = df_feed_enriched.copy()
        # Evaluate exits
        for token, pos in list(open_positions.items()):
            if pos["qty"] <= 0:
                continue
            row = snapshot.loc[snapshot["Token"]==token]
            if row.empty:
                continue
            row = row.iloc[0]
            signal = should_exit_position(token, row, pos, cfg)
            if signal:
                if cfg.dry_run:
                    print(f"[EXIT-SIM] {signal}")
                else:
                    # Place live exit order (opposite side, qty=pos['qty'])
                    # Not implemented in this clean run-all; follow same pattern as place_orders()
                    print(f"[EXIT-LIVE] Would place exit for {token}: {signal}")
                # After exit, mark position closed
                open_positions[token] = {"qty": 0, "side": pos["side"], "avg_price": pos["avg_price"]}

def start_exit_manager(cfg: ExitConfig):
    global _exit_thread, _exit_stop_evt
    _exit_stop_evt = threading.Event()
    _exit_thread = threading.Thread(target=_exit_worker, args=(cfg,), daemon=True)
    _exit_thread.start()

def stop_exit_manager():
    global _exit_thread, _exit_stop_evt
    _exit_stop_evt.set()
    if _exit_thread is not None:
        _exit_thread.join(timeout=5)

def place_orders(plan: Dict[str, Any], df_enriched: pd.DataFrame, dry_run: bool = True) -> List[Dict[str, Any]]:
    results = []
    if not plan or "legs" not in plan:
        return results

    order_api = None
    if not dry_run:
        if not UPSDK_AVAILABLE:
            raise RuntimeError("Upstox SDK not available for live orders")
        if not CredentialUpstox.ACCESS_TOKEN:
            raise RuntimeError("ACCESS_TOKEN missing for live orders")
        configuration = upstox_client.Configuration()
        configuration.access_token = CredentialUpstox.ACCESS_TOKEN
        api_client = upstox_client.ApiClient(configuration)
        order_api = upstox_client.OrderApi(api_client)

    for leg in plan["legs"]:
        token = str(leg["token"])
        row = df_enriched.loc[df_enriched["Token"]==token]
        if row.empty:
            results.append({"status":"rejected","reason":"token_not_found","leg":leg})
            continue
        row = row.iloc[0]
        lot = int(row["lot_size"]) if pd.notna(row["lot_size"]) else 0
        qty = int(leg["qty"])
        if lot and qty % lot != 0:
            results.append({"status":"rejected","reason":f"qty_not_multiple_of_lot({lot})","leg":leg})
            continue
        if qty <= 0 or qty > MAX_QTY_PER_LEG:
            results.append({"status":"rejected","reason":"qty_bounds","leg":leg})
            continue

        side = leg["side"]
        product_code = PRODUCT_MAP.get(leg.get("product", DEFAULT_PRODUCT), PRODUCT_MAP[DEFAULT_PRODUCT])
        order_type = leg.get("order_type","MARKET")
        price = leg.get("price", None)
        # Use current Mid price as simulated fill
        fill_price = float(row.get("Mid")) if not np.isnan(row.get("Mid", np.nan)) else float(row.get("Ltp", 0.0))

        if dry_run:
            results.append({"status":"simulated","token":token,"qty":qty,"side":side,"product":product_code,"order_type":order_type,"fill_price":fill_price})
            record_fill({"token": token, "side": side, "qty": qty, "price": fill_price})
        else:
            # LIVE order request
            req = upstox_client.PlaceOrderRequest(
                quantity=str(qty),
                product=product_code,
                validity="DAY",
                price=float(price) if (price is not None and order_type=="LIMIT") else 0.0,
                tag="LLM-STRATEGY",
                instrument_token=token,
                order_type=order_type,
                transaction_type=side,
                disclosed_quantity=0,
                trigger_price=0.0,
                is_amo=False
            )
            resp = order_api.place_order(body=req, api_version="3.0")
            results.append({"status":"placed","order_id":resp.data.order_id,"token":token,"qty":qty,"side":side})
            # For now assume marketable order gets filled near current price
            record_fill({"token": token, "side": side, "qty": qty, "price": fill_price})
    return results


In [17]:

# ---- Orchestration ----

# 1) Select strikes around ATM
if SIMULATION_MODE:
    underlying_ltp = float(df_chain["strike_price"].median())
else:
    underlying_ltp = init_ltp if init_ltp is not None else float(df_chain["strike_price"].median())

strike_list = nearest_strikes(underlying_ltp, UNDERLYING, span=SPAN_STRIKES)
df_chain_sel = df_chain[df_chain["strike_price"].isin(strike_list)].sort_values(["strike_price","instrument_type"])
token_list = df_chain_sel["instrument_key"].dropna().astype(str).unique().tolist()

assert token_list, "No tokens to subscribe after strike filtering"
print(f"Selected strikes for {UNDERLYING}: {sorted(set(strike_list))}")
print(f"Tokens selected: {len(token_list)}")

display(df_chain_sel.head(10))

# 2) Start feed (simulation or live)
if SIMULATION_MODE:
    sim = SimStreamer(token_list, step=0.8, period_s=0.5, run_seconds=RUN_DEMO_SECONDS)
    sim.start()
else:
    live_streamer = start_live_stream(token_list, mode=WEBSOCKET_MODE)

# 3) Wait a moment for feed to populate
time.sleep(2.0)

# 4) Snapshot & plan
with _df_lock:
    snapshot = df_feed_enriched.copy()

print("Feed snapshot rows:", len(snapshot))
display(snapshot.tail(6))

plan = ask_llm_for_strategy(snapshot, use_mock=(not USE_LLM))
print("Received plan:", json.dumps(plan, indent=2))

# 5) Validate
try:
    validated = validate_strategy(plan, snapshot)
    print("Plan validated.")
except Exception as e:
    validated = None
    print("Validation failed:", e)

# 6) Place orders (dry-run by default)
orders = []
if validated:
    orders = place_orders(validated, snapshot, dry_run=(not ORDERS_LIVE))
    print("Order results:")
    print(json.dumps(orders, indent=2))

# 7) Start exit manager
cfg = ExitConfig(dry_run=(not EXIT_MANAGER_LIVE))
start_exit_manager(cfg)

# 8) Allow exits to evaluate during the demo
if SIMULATION_MODE:
    time.sleep(max(0, RUN_DEMO_SECONDS - 2))  # let the sim keep streaming


Selected strikes for NIFTY: [24850, 24900, 24950, 25000, 25050]
Tokens selected: 10


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,freeze_quantity,exchange_token,minimum_lot,tick_size,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,asset_key,underlying_key,last_trading_date,price_quote_unit
16851,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47711,75.0,1800.0,47711,75.0,5.0,INDEX,INDEX,NIFTY 24850 CE 23 SEP 25,24850.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16850,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47712,75.0,1800.0,47712,75.0,5.0,INDEX,INDEX,NIFTY 24850 PE 23 SEP 25,24850.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16849,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47717,75.0,1800.0,47717,75.0,5.0,INDEX,INDEX,NIFTY 24900 CE 23 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16848,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47718,75.0,1800.0,47718,75.0,5.0,INDEX,INDEX,NIFTY 24900 PE 23 SEP 25,24900.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16844,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47723,75.0,1800.0,47723,75.0,5.0,INDEX,INDEX,NIFTY 24950 CE 23 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16847,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47724,75.0,1800.0,47724,75.0,5.0,INDEX,INDEX,NIFTY 24950 PE 23 SEP 25,24950.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16876,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47733,75.0,1800.0,47733,75.0,5.0,INDEX,INDEX,NIFTY 25000 CE 23 SEP 25,25000.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16875,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47734,75.0,1800.0,47734,75.0,5.0,INDEX,INDEX,NIFTY 25000 PE 23 SEP 25,25000.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16865,1.0,NSE_FO,NIFTY,NSE,2025-09-23,CE,NIFTY,NIFTY,NSE_FO|47751,75.0,1800.0,47751,75.0,5.0,INDEX,INDEX,NIFTY 25050 CE 23 SEP 25,25050.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN
16864,1.0,NSE_FO,NIFTY,NSE,2025-09-23,PE,NIFTY,NIFTY,NSE_FO|47752,75.0,1800.0,47752,75.0,5.0,INDEX,INDEX,NIFTY 25050 PE 23 SEP 25,25050.0,1.0,NaN,NaN,NaN,NSE_INDEX|Nifty 50,NSE_INDEX|Nifty 50,NaN,NaN


TypeError: MarketDataStreamerV3.__init__() got an unexpected keyword argument 'instrument_key'

In [ ]:

# ---- Shutdown / cleanup ----
if SIMULATION_MODE:
    sim.stop()
    sim.join(timeout=3)

stop_exit_manager()

# Final state
print("Open positions:")
print(open_positions)
with _df_lock:
    final_snapshot = df_feed_enriched.copy()
display(final_snapshot.tail(8))
print("Notebook run complete.")


Open positions:
{'NSE_FO|60011': {'qty': 50, 'side': 'SELL', 'avg_price': 24054.22}, 'NSE_FO|60014': {'qty': 50, 'side': 'SELL', 'avg_price': 24132.18}}


,Token,Ltp,Ltq,Cp,BidP1,BidQ1,AskP1,AskQ1,Ltt,Oi,Iv,Atp,Tbq,Tsq,instrument_key,lot_size,trading_symbol,strike_price,tick_size,instrument_type,expiry,name,Mid,Spread
2,NSE_FO|60007,24102.13,150.0,38.13,24101.78,600.0,24102.48,450.0,2025-09-19 09:18:40.955000+05:30,250000.0,0.20708,24109.81,105161.0,427870.0,NSE_FO|60007,50.0,NIFTY 23950 CE 25 SEP 25,23950.0,0.05,CE,2025-09-25,NIFTY,24102.13,0.7
3,NSE_FO|60008,24221.39,225.0,76.50,24221.04,600.0,24221.74,450.0,2025-09-19 09:18:40.961000+05:30,250000.0,0.16984,24230.72,478105.0,153768.0,NSE_FO|60008,50.0,NIFTY 23950 PE 25 SEP 25,23950.0,0.05,PE,2025-09-25,NIFTY,24221.39,0.7
4,NSE_FO|60009,23882.49,150.0,2.23,23882.14,600.0,23882.84,150.0,2025-09-19 09:18:40.966000+05:30,500000.0,0.26057,23877.24,429262.0,479067.0,NSE_FO|60009,50.0,NIFTY 24000 CE 25 SEP 25,24000.0,0.05,CE,2025-09-25,NIFTY,23882.49,0.7
5,NSE_FO|60010,23842.56,75.0,2.01,23842.21,900.0,23842.91,900.0,2025-09-19 09:18:40.970000+05:30,100000.0,0.16048,23839.15,194827.0,257874.0,NSE_FO|60010,50.0,NIFTY 24000 PE 25 SEP 25,24000.0,0.05,PE,2025-09-25,NIFTY,23842.56,0.7
6,NSE_FO|60011,24053.96,225.0,38.15,24053.61,300.0,24054.31,900.0,2025-09-19 09:18:40.974000+05:30,250000.0,0.14115,24053.22,89600.0,309791.0,NSE_FO|60011,50.0,NIFTY 24050 CE 25 SEP 25,24050.0,0.05,CE,2025-09-25,NIFTY,24053.96,0.7
7,NSE_FO|60012,23578.62,150.0,-93.76,23578.27,600.0,23578.97,900.0,2025-09-19 09:18:40.978000+05:30,100000.0,0.12179,23570.64,69319.0,430481.0,NSE_FO|60012,50.0,NIFTY 24050 PE 25 SEP 25,24050.0,0.05,PE,2025-09-25,NIFTY,23578.62,0.7
8,NSE_FO|60013,24147.85,150.0,-89.80,24147.50,150.0,24148.20,900.0,2025-09-19 09:18:40.981000+05:30,500000.0,0.22535,24155.27,32834.0,69313.0,NSE_FO|60013,50.0,NIFTY 24100 CE 25 SEP 25,24100.0,0.05,CE,2025-09-25,NIFTY,24147.85,0.7
9,NSE_FO|60014,24129.50,150.0,8.29,24129.15,450.0,24129.85,300.0,2025-09-19 09:18:40.985000+05:30,250000.0,0.24987,24134.41,153537.0,73414.0,NSE_FO|60014,50.0,NIFTY 24100 PE 25 SEP 25,24100.0,0.05,PE,2025-09-25,NIFTY,24129.50,0.7


Notebook run complete.



## Notes & reminders

- **Switch to live**: set `SIMULATION_MODE=False`, ensure `upstox_client` is installed and `CredentialUpstox.ACCESS_TOKEN` is set.
- **Live orders**: set `ORDERS_LIVE=True` only after you've verified the plan, validation, and product codes.
- **Exit manager**: uses **Mid** when available; adjust `target_pct/stop_pct` in `ExitConfig`.
- **Risk controls**: also guard notional exposure & concurrency outside of this demo.
- **Persistence**: snapshot `df_feed_enriched` to parquet periodically for audit.
- **Compliance**: confirm tick sizes, lot sizes, and product codes with exchange & broker docs in your region.
